# Candlestick Chat for Stock

In [ ]:
from flask import Flask
from flask import render_template, request
from flask_sqlalchemy import SQLAlchemy
import pymysql
#import flask_sqlalchemy
import config
import os
import json

#from sqlalchemy import create_engine
#from sqlalchemy.orm import scoped_session, sessionmaker
#from sqlalchemy.ext.declarative import declarative_bas


import random
import datetime
import pymysql
import pandas as pd

from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.charts import Kline
from pyecharts import options as opts
#import pyecharts.events as events
#from pyecharts_javascripthon.dom import alert

app = Flask(__name__)
app.config.from_object(config)
db = SQLAlchemy(app)

db.init_app(app)

class transactions(db.Model):
    __tablename__ = 'transactions' 
    sid  = db.Column(db.String(4), primary_key = True)
    date = db.Column(db.Date, primary_key =True)
    capacity = db.Column(db.Integer)
    turnover = db.Column(db.BigInteger)
    open = db.Column(db.Float)
    high = db.Column(db.Float)
    low = db.Column(db.Float)
    close = db.Column(db.Float)
    transaction = db.Column(db.Integer)
    
    def __init__(self, sid, date, capacity, turnover, open, high, low, close, transaction):
        self.sid = sid
        self.city = date
        self.addr = capacity
        self.pin = turnover
        self.open= open
        self.high= high
        self.low= low
        self.close = close
        self.transaction= transaction

def kline_base(Date, KBar, sid):
    kline = Kline()    
    kline.add_xaxis(xaxis_data = Date) 
    kline.add_yaxis(series_name = "日K線", 
                    y_axis = KBar,  
                    itemstyle_opts = opts.ItemStyleOpts(color = "#C23531",
                                                        color0 = "#00da3c", 
                                                        border_color = "#C23531", 
                                                        border_color0 = "#00da3c"),
                    )
    kline.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                             tooltip_opts = opts.TooltipOpts(trigger="axis",
                                                             trigger_on = 'click',
                                                             axis_pointer_type="cross", 
                                                             background_color="rgba(245, 245, 245, 0.8)",
                                                             border_width=1,
                                                             border_color="#ccc",
                                                             textstyle_opts=opts.TextStyleOpts(color="#000"), 
                                                             ),                      
                        )
    kline.set_global_opts(title_opts = opts.TitleOpts(title = sid, subtitle ='股價'),
                              xaxis_opts = opts.AxisOpts(is_scale = True),
                              yaxis_opts = opts.AxisOpts(is_scale = True, splitarea_opts = opts.SplitAreaOpts(is_show = True, areastyle_opts = opts.AreaStyleOpts(opacity=1))),    
                              datazoom_opts = [opts.DataZoomOpts(type_ = "inside"),   
                                               opts.DataZoomOpts(is_show=True,  
                                                                 type_="slider",),
                                               ],
                              tooltip_opts = opts.TooltipOpts(trigger="axis",
                                                              axis_pointer_type="cross", 
                                                              background_color="rgba(245, 245, 245, 0.8)",
                                                              border_width=1,
                                                              border_color="#ccc",
                                                              textstyle_opts=opts.TextStyleOpts(color="#000"), 
                                                              ),
                              )

    return kline


    
@app.route('/Kline', methods=['GET', 'POST'])
def get_kline_chart():
    #connect_mysql()
    if request.method == 'POST':
        global sid
        sid = request.form.get('sid')
    query = transactions.query.filter(transactions.sid==sid).all()
      
    KBar = [] 
    Date = []
     
    for _query in query:
        Date.append(_query.date)
        KBar.append([_query.open, _query.close, _query.low, _query.high])
        kline=kline_base(Date, KBar, sid)
        #print(_query.date)
        #print([_query.open, _query.close, _query.low, _query.high])
        #print(kline.dump_options())
    return kline.dump_options()

@app.route("/", methods=['GET', 'POST'])
def index():
    return render_template("index.html")

@app.route('/Dataset', methods=['GET', 'POST'])
def get_dataset():
    if request.method == 'POST':
        #sid = request.form.get('sid')
        start = request.form.get('start')
        end =  request.form.get('end')
        ctype =  request.form.get('classtype')
    query = transactions.query.filter(transactions.sid==sid).filter(transactions.date.between(start, end)).all()

    KBar = [] 
    Date = []
    
    for _query in query:
        Date.append(_query.date)
        KBar.append([_query.open, _query.close, _query.low, _query.high])
        kline=kline_base(Date, KBar, sid)

    #save json file        
    jsonpath='dataset'
    
    if not os.path.exists(jsonpath):
        os.mkdir(jsonpath)
    
    dataset={"sid":sid,'KBar':KBar,"ctype":ctype}
    print(dataset)
    
    filename='{}_{}_{}.json'.format(sid, start, end)
    #print(filename)
    with open(jsonpath+'/'+filename, 'w') as f:
        json.dump(dataset, f)

    return kline.dump_options()  
        
        
if __name__ == '__main__':   
    #app.run(debug=True)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


2020-06-09 22:44:55,540 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2020-06-09 22:44:55,543 INFO sqlalchemy.engine.base.Engine {}
2020-06-09 22:44:55,547 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2020-06-09 22:44:55,547 INFO sqlalchemy.engine.base.Engine {}
2020-06-09 22:44:55,552 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2020-06-09 22:44:55,552 INFO sqlalchemy.engine.base.Engine {}
2020-06-09 22:44:55,554 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2020-06-09 22:44:55,555 INFO sqlalchemy.engine.base.Engine {}
2020-06-09 22:44:55,558 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2020-06-09 22:44:55,559 INFO sqlalchemy.engine.base.Engine {}
2020-06-09 22:44:55,560 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2020-06-09 22:44:55,561 INFO sqlalchemy.engine.base.E

127.0.0.1 - - [09/Jun/2020 22:44:55] "POST /Dataset HTTP/1.1" 200 -
